## **_Backtesting Template_**

**_A step by step walkthrough of the backtesting process. Shouldn't be touched._**

### **_Libraries_**

In [1]:
# Backtesting
import vectorbt as vbt

# Data
import yfinance as yf
import pandas as pd
import numpy as np

# Time
import datetime as dt

# Plotting
import plotly.graph_objects as go
from plotly.subplots import make_subplots

### **_Data_**

In [2]:
def Data(symbols, interval, start, end):
    """
    ## Data

    ### Description:
    This function collects close data from Yahoo Finance
    in a structure that agrees with the VectorBT Ind
    Factory function.

    ### Args:
        - symbols (List): List of symbol's data to collect
        - period (Integer): Number of most recent days to data to collect
        - interval (String): Timeframe of data to collect

    ### Returns:
        - price (Dataframe): VectorBT agreaeble dataframe of price data
    """

    # Time periods
    end_time = end
    start_time = start

    # Downloading close data
    price = vbt.YFData.download(symbols=symbols,
                                missing_index="drop",
                                start=start_time,
                                end=end_time,
                                interval=interval).get("Close")
    
    # Returning the close prices
    return price

### **_Strategy_**

In [3]:
def Custom_Indicator(close, moving_average_win=21, standard_deviation_mag=1):
    """
    ## Custom Indicator

    ### Note:
    Must be changed to suit specific strategy

    ### Description:
    Basic template for an indicator that is compatable with
    VectorBT.

    ### Args:
        - close (Dataframe): Pandas dataframe of close data
        - moving_average_win (int): It's in the name
        - standard_deviation_mag (int): The coefficient for the standard deviation

    ### Returns:
        - signals (Array): Numpy array of all signals

    ### Note:
    'Signals' var should be a 1*n dimensional array of no. 
    of positions to buy or sell or neither.
    """

    # Interchangable depending on strategy vvv
    Moving_Average = close.rolling(moving_average_win).mean()
    Standard_Deviation = close.rolling(moving_average_win).std()

    # Interchangable depending on strategy vvv
    Upper_Band = Moving_Average + (standard_deviation_mag * Standard_Deviation)
    Lower_Band = Moving_Average - (standard_deviation_mag * Standard_Deviation)
    
    # Interchangable depending on strategy vvv
    signals = np.where((close < Lower_Band), 1, 0)
    signals = np.where((close > Upper_Band), -1, signals) 

    return signals

### **_Backtesting_**

In [4]:
def Backtest(ind, price, moving_average_win, standard_deviation_mag, cash):
    """
    ## Backtest

    ### Description:
    A function to run a backtest of your strategy on an array of parameters. A
    portfolio object of the backtest is then returned.

    ### Args:
        - ind (VectorBT Ind): A VectorBT indicator factory object
        - price (Dataframe): A Dataframe of the chosen symbol's price data
        - moving_average_win (List): A list of all vars to try on this parameter
        - standard_deviation_mag (List): A list of all vars to try on this parameter
        - cash (Integer): Amount of starting cash for your portfolio

    ### Returns:
        - signals (Object): Object of signals and various attributes of backtest
        - portfolio (Object): Object of portfolios for each backtest
    """

    # Extracting Signals
    signals = ind.run(Price=price,
                      Moving_Average_Win=moving_average_win,
                      Standard_Deviation_Mag=standard_deviation_mag,
                      param_product=True)
    
    # Creating a portfolio
    portfolio = vbt.Portfolio.from_orders(close=price,
                                          size_type="Amount",
                                          size=signals.Output, # The attribute depends
                                          init_cash=cash,
                                          freq="D")
    
    # Returning signals and portfolio
    return signals, portfolio

### **_Plotting_**

In [5]:
def Plotting(portfolio, signals, set):
    """
    ## Plotting

    ### Note:
    Must be changed to suit specific strategy

    ### Decription:
    A function to plot the trades of a specific backtest and position status throughout.
    Also plots indicators of chosen strategy.

    ### Args:
        - portfolio (VectorBT Portfolio object): Portfolio of specific backtest
        - signals (Object): Object of various dataframes of data and signals derived from said data
        - set (Tuple): A tuple of parameters of the backtest you want to plot

    ### Returns:
        - None
    """
    
    # Extracting specific set of trades for specific backtest set
    trades = portfolio.orders.records_readable[portfolio.orders.records_readable["Column"]==set]
    trades["Colour"] = trades["Side"].apply(lambda side: "green" if side == "Buy" else "red")
    trades["Marker"] = trades["Side"].apply(lambda side: "triangle-up" if side == "Buy" else "triangle-down")

    # Recalculating indicators for plotting
    data = pd.DataFrame({"Close":signals.Price[set]}, index=signals.Price[set].index)
    data["Moving Average"] = data["Close"].rolling(set[0]).mean()
    data["Standard Deviation"] = data["Close"].rolling(set[0]).std()
    data["Upper Band"] = data["Moving Average"] + (set[1] * data["Standard Deviation"])
    data["Lower Band"] = data["Moving Average"] - (set[1] * data["Standard Deviation"])

    # Calculating position size throughout backtest
    pos_count_current = 0
    pos_colour = []
    pos_count = []
    for i in range(len(signals.Output[set])):
        pos_count_current += signals.Output[set].iloc[i]
        pos_count.append(pos_count_current)
        if pos_count_current < 0:
            pos_colour.append("red")
        elif pos_count_current > 0:
            pos_colour.append("green")
        else:
            pos_colour.append("blue")

    # Creating subplots
    fig = make_subplots(rows=2, cols=1,
                        shared_xaxes=True,
                        row_heights=[0.75, 0.25],
                        vertical_spacing=0.05)

    # Plotting the close price
    fig.add_trace(go.Scatter(x=data.index,
                             y=data["Close"],
                             mode="lines",
                             name="Price"), row=1, col=1)
    
    # Plotting the moving average
    fig.add_trace(go.Scatter(x=data.index,
                             y=data["Moving Average"],
                             mode="lines",
                             name="Moving Average"), row=1, col=1)
    
    # Plotting the upper band
    fig.add_trace(go.Scatter(x=data.index,
                             y=data["Upper Band"],
                             mode="lines",
                             name="Upper Band"), row=1, col=1)
    
    # Plotting the lower band
    fig.add_trace(go.Scatter(x=data.index,
                             y=data["Lower Band"],
                             mode="lines",
                             name="Lower Band"), row=1, col=1)
    
    # Plotting the trades
    fig.add_trace(go.Scatter(x=trades["Timestamp"],
                             y=trades["Price"],
                             mode="markers",
                             marker=dict(color=trades["Colour"], symbol=trades["Marker"], size=10),
                             name="Buys + Sells"), row=1, col=1)
    
    # Plotting position 
    fig.add_trace(go.Scatter(x=data.index,
                             y=pos_count,
                             mode="markers",
                             marker=dict(color=pos_colour, size=2),
                             name="Position"), row=2, col=1)

    # Updating the layout
    fig.update_layout(title=dict(text=f"Backtest: {set}", font=dict(color="white")),
                      height=700,
                      # Changing colors of plot surroundings and grid respectively
                      paper_bgcolor="rgba(70,70,70,1)",
                      plot_bgcolor="rgba(230,230,230,1)",
                      # Updating axes settings
                      xaxis=dict(tickfont=dict(color="white"),
                                 titlefont=dict(color="white"),
                                 gridcolor="rgba(0,0,0,0.1)", gridwidth=2),
                      xaxis2=dict(title="Date & Time",
                                 tickfont=dict(color="white"),
                                 titlefont=dict(color="white"),
                                 gridcolor="rgba(0,0,0,0.1)", gridwidth=2),
                      yaxis=dict(title="Price",
                                 tickfont=dict(color="rgba(230,230,230,1)"),
                                 titlefont=dict(color="rgba(230,230,230,1)"),
                                 gridcolor="rgba(0,0,0,0.1)", gridwidth=2),
                      yaxis2=dict(title="Position Size",
                                 tickfont=dict(color="white"),
                                 titlefont=dict(color="white"),
                                 gridcolor="rgba(0,0,0,0.1)", gridwidth=2),
                      # Creating frames of the top and bottom plot respectively
                      shapes = [go.layout.Shape(type="rect",
                                                xref="paper",
                                                yref="paper",
                                                x0=0,
                                                y0=0.29,
                                                x1=1,
                                                y1=1,
                                                line={'width': 3, 'color': 'black'}),
                                go.layout.Shape(type="rect",
                                                xref="paper",
                                                yref="paper",
                                                x0=0,
                                                y0=0,
                                                x1=1,
                                                y1=0.24,
                                                line={'width': 3, 'color': 'black'})],
                      # Updating the legend's font color
                      legend=dict(font=dict(color="rgba(230,230,230,1)")))

    # Displaying
    fig.show()

### **_Workspace_**

In [6]:
price = Data(["AAPL", "AMZN", "PLTR"], "1m", dt.datetime(year=2024, month=2, day=6), dt.datetime(year=2024, month=2, day=7))

In [7]:
ind = vbt.IndicatorFactory(
    class_name="BBands Strategy",                                 # Identifier
    short_name="BBands",                                          # Identifier
    input_names=["Price"],                                        # Inputs refer to price data
    param_names=["Moving_Average_Win", "Standard_Deviation_Mag"], # These are the settings that can be changed
    output_names=["Output"]                                       # Attribute to access data and signals derived from said data
    ).from_apply_func(Custom_Indicator,                           # Reference to original indicator
                      Moving_Average_Win=21,                      # Default setting
                      Standard_Deviation_Mag=1,                   # Default setting
                      keep_pd=True)                               # Retains Input in pandas structure

In [8]:
Signals, Portfolio = Backtest(ind, price, [21, 28, 35, 42], [1, 1.5, 2], 10000)

In [15]:
Portfolio.stats()

C:\Users\redmo\AppData\Local\Temp\ipykernel_12096\300726787.py:1: UserWarning: Object has multiple columns. Aggregating using <function mean at 0x000001C07E69E290>. Pass column to select a single column/group.
  Portfolio.stats()


Start                           2024-02-06 14:30:00+00:00
End                             2024-02-06 20:59:00+00:00
Period                                  389 days 00:00:00
Start Value                                       10000.0
End Value                                    10014.146924
Total Return [%]                                 0.141469
Benchmark Return [%]                             2.838464
Max Gross Exposure [%]                         112.874148
Total Fees Paid                                       0.0
Max Drawdown [%]                                 0.245108
Max Drawdown Duration                   152 days 00:40:00
Total Trades                                    44.777778
Total Closed Trades                             43.888889
Total Open Trades                                0.888889
Open Trade PnL                                   0.479414
Win Rate [%]                                    75.995592
Best Trade [%]                                   1.714355
Worst Trade [%

In [16]:
Portfolio.total_profit().idxmax()

(42, 1.0, 'AAPL')

In [17]:
Portfolio.stats(column=(35, 2.0, "AMZN"))

Start                           2024-02-06 14:30:00+00:00
End                             2024-02-06 20:59:00+00:00
Period                                  389 days 00:00:00
Start Value                                       10000.0
End Value                                    10021.368439
Total Return [%]                                 0.213684
Benchmark Return [%]                            -0.797654
Max Gross Exposure [%]                          35.304612
Total Fees Paid                                       0.0
Max Drawdown [%]                                 0.118763
Max Drawdown Duration                   117 days 00:00:00
Total Trades                                           21
Total Closed Trades                                    20
Total Open Trades                                       1
Open Trade PnL                                   8.311246
Win Rate [%]                                         95.0
Best Trade [%]                                   0.764399
Worst Trade [%

In [18]:
Plotting(Portfolio, Signals, (35, 2.0, "AAPL"))